In [85]:
import sys
import os
import pandas as pd
from pathlib import Path


In [86]:
# Añadir el directorio src al path de Python
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


In [87]:
from calculadora_margen.encoder import Encoder
from calculadora_margen.cleaning.cleaner_df import DataFrameCleaner
from calculadora_margen.cleaning.params import Parameters
from calculadora_margen.cleaning.validador import Validator
from calculadora_margen.cleaning.outliers_manager import OutliersManager

In [88]:
project_root_path = Path(src_path).parent
data_path = project_root_path / 'data'
raw_path = data_path / 'raw'
clean_path = data_path / 'clean'

ETL master_lotes

In [89]:
master_lotes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [90]:
cleaner = DataFrameCleaner(master_lotes)
params = Parameters.master_lotes

master_lotes = (cleaner
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .rows_cleaner.drop_duplicates()
    .rows_cleaner.drop_na(params.drop_na_subset)
    .data_cleaner.to_upper()
    .get_df()
)


=== SELECCIÓN Y RENOMBRADO DE COLUMNAS ===
  columnas_iniciales: 16
  columnas_finales: 3
  columnas_conservadas: ['Cód. artículo', 'LOTE', 'LOTEINTERNO']
  columnas_renombradas: {'Cód. artículo': 'articulo', 'LOTE': 'lote_proveedor', 'LOTEINTERNO': 'lote_componente'}

=== ELIMINACIÓN DE DUPLICADOS ===
  filas_eliminadas: 70
  columnas_consideradas: todas
  criterio: mantener_first
  Tamaño final del DataFrame: 19256

=== ELIMINACIÓN DE VALORES NA ===
  filas_eliminadas: 4
  columnas_consideradas: ['lote_componente']
  Tamaño final del DataFrame: 19252

=== CONVERSIÓN A MAYÚSCULAS ===
  columnas_procesadas: ['articulo', 'lote_proveedor', 'lote_componente']


In [91]:
validator = Validator(master_lotes)
master_lotes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

In [92]:
# Creamos clave única para poder hacer merge en otros df
encoder = Encoder(master_lotes)
master_lotes = encoder.create_key(col1='articulo', col2='lote_proveedor', new_col_name='clave_merge')

In [93]:
duplicados = master_lotes['clave_merge'].duplicated().sum()
duplicados

np.int64(0)

In [94]:
master_lotes.sample(2)

,articulo,lote_proveedor,lote_componente,clave_merge
14333,MAT348,FS026,2310-052,MAT348-FS026
9226,SEM101,201222,201222,SEM101-201222


In [95]:
master_lotes.to_csv(clean_path / 'master_lotes_clean.csv', index=False)

ETL costes

In [96]:
costes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [97]:
costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Cód. almacén estructura  8047 non-null   object
 1   DESCALM                  8047 non-null   object
 2   Cód. artículo            19326 non-null  object
 3   Artículo                 19326 non-null  object
 4   FECDOC                   8047 non-null   object
 5   LOTE                     19326 non-null  object
 6   FECCADUC                 8047 non-null   object
 7   LOTEINTERNO              19322 non-null  object
 8   UNIDADES                 8047 non-null   object
 9   PRCMONEDA                8047 non-null   object
 10  % descuento 1            8047 non-null   object
 11  TIPDOC                   8047 non-null   object
 12  NUMDOC                   8047 non-null   object
 13  REFERENCIA               8043 non-null   object
 14  Cód. proveedor           8047 non-null

In [98]:
cleaner = DataFrameCleaner(costes)
params = Parameters.costes

costes = (cleaner
    .rows_cleaner.drop_na(params.drop_na_subset)
    .rows_cleaner.drop_duplicates()
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .data_cleaner.fix_numeric_format(params.cols_to_float)
    .rows_cleaner.drop_duplicates_batch(params.drop_duplicates_subset)
    .data_cleaner.to_upper()
    .get_df()
)


=== ELIMINACIÓN DE VALORES NA ===
  filas_eliminadas: 11279
  columnas_consideradas: ['PRCMONEDA']
  Tamaño final del DataFrame: 8047

=== ELIMINACIÓN DE DUPLICADOS ===
  filas_eliminadas: 1
  columnas_consideradas: todas
  criterio: mantener_first
  Tamaño final del DataFrame: 8046

=== SELECCIÓN Y RENOMBRADO DE COLUMNAS ===
  columnas_iniciales: 16
  columnas_finales: 3
  columnas_conservadas: ['Cód. artículo', 'PRCMONEDA', 'LOTEINTERNO']
  columnas_renombradas: {'Cód. artículo': 'componente', 'PRCMONEDA': 'coste_componente_unitario', 'LOTEINTERNO': 'lote_componente'}

=== CORRECCIÓN DE FORMATO NUMÉRICO ===
  columnas_procesadas: ['coste_componente_unitario']

=== ELIMINACIÓN DE DUPLICADOS POR LOTE ===
  columna: lote_componente
  filas_eliminadas: 46
  Tamaño final del DataFrame: 8000

=== CONVERSIÓN A MAYÚSCULAS ===
  columnas_procesadas: ['componente', 'lote_componente']


In [99]:
validator = Validator(costes)

costes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)


=== RESUMEN DE VALIDACIÓN ===
Tamaño inicial del DataFrame: 8000

Filas inválidas por columna:
  - componente: 1 filas
  - lote_componente: 21 filas

Tamaño final del DataFrame: 7978
Total filas eliminadas: 22


In [100]:
# Ver las filas inválidas para una columna específica
#invalid_rows = validator.get_invalid('lote_interno')
#print(invalid_rows.head(10))

In [101]:
outliers_manager = OutliersManager(costes)

costes = (outliers_manager
    .process_outliers()
    .clean_columns()
    .get_df()
)


=== RESUMEN DE OUTLIERS ===
Outliers detectados inicialmente: 71
Outliers reemplazados por la media: 66
Outliers restantes: 5


In [102]:
costes.to_csv(clean_path / 'costes_clean.csv', index=False)

In [103]:
costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7978 entries, 0 to 7977
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   coste_componente_unitario  7978 non-null   float64
 1   lote_componente            7978 non-null   object 
 2   componente                 7978 non-null   object 
dtypes: float64(1), object(2)
memory usage: 187.1+ KB


ETL fabricaciones

In [104]:
fabricaciones = pd.read_csv(raw_path / 'fabricaciones_2025.csv',  encoding='UTF-8', sep=';', dtype=str)

In [105]:
# No queremos trabajar con lote_componente_proveedor, unimos a master_lotes para obtener lote_componente
encoder = Encoder(fabricaciones)

fabricaciones = encoder.create_key(col1='Componente', col2='Lote Componente', new_col_name='clave_merge')
fabricaciones = fabricaciones.merge(master_lotes, on="clave_merge", how="left")

In [106]:
cleaner = DataFrameCleaner(fabricaciones)
params = Parameters.fabricaciones

fabricaciones = (cleaner
    .rows_cleaner.drop_duplicates()
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .data_cleaner.fix_numeric_format(params.cols_to_float)
    .rows_cleaner.drop_na(params.drop_na_subset)
    .data_cleaner.to_upper()
    .get_df()
)


=== ELIMINACIÓN DE DUPLICADOS ===
  filas_eliminadas: 0
  columnas_consideradas: todas
  criterio: mantener_first
  Tamaño final del DataFrame: 19659

=== SELECCIÓN Y RENOMBRADO DE COLUMNAS ===
  columnas_iniciales: 21
  columnas_finales: 9
  columnas_conservadas: ['Fecha Recepción', 'Producto', 'Lote Producto', 'Unidades Fabricadas', 'Componente', 'lote_componente', 'Consumo Unitario', 'Consumo Total', 'Nº Orden']
  columnas_renombradas: {'Fecha Recepción': 'fecha_fabricacion', 'Producto': 'articulo', 'Lote Producto': 'lote_articulo', 'Componente': 'componente', 'Consumo Unitario': 'consumo_unitario', 'Consumo Total': 'consumo_total', 'Unidades Fabricadas': 'unidades_fabricadas', 'Nº Orden': 'id_orden'}

=== CORRECCIÓN DE FORMATO NUMÉRICO ===
  columnas_procesadas: ['unidades_fabricadas', 'consumo_unitario', 'consumo_total']

=== ELIMINACIÓN DE VALORES NA ===
  filas_eliminadas: 76
  columnas_consideradas: ['lote_articulo', 'lote_componente']
  Tamaño final del DataFrame: 19583

=== 

In [61]:
fabricaciones.to_csv(clean_path / 'fabricaciones_clean.csv', index=False)

In [107]:
# Unimos df de costes y fabricaciones
encoder = Encoder(fabricaciones)
fabricaciones = encoder.create_key(col1='componente', col2='lote_componente', new_col_name='clave_merge')

encoder = Encoder(costes)
costes = encoder.create_key(col1='componente', col2='lote_componente', new_col_name='clave_merge')

fabricaciones_costes = fabricaciones.merge(costes, on="clave_merge", how="left")

In [108]:
fabricaciones_costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19583 entries, 0 to 19582
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   fecha_fabricacion          19583 non-null  object 
 1   articulo                   19583 non-null  object 
 2   lote_articulo              19583 non-null  object 
 3   unidades_fabricadas        19583 non-null  float64
 4   componente_x               19583 non-null  object 
 5   lote_componente_x          19583 non-null  object 
 6   consumo_unitario           19583 non-null  float64
 7   consumo_total              19583 non-null  float64
 8   id_orden                   19583 non-null  object 
 9   clave_merge                19583 non-null  object 
 10  coste_componente_unitario  16622 non-null  float64
 11  lote_componente_y          16622 non-null  object 
 12  componente_y               16622 non-null  object 
dtypes: float64(4), object(9)
memory usage: 1.9+ MB

In [109]:
columnas_a_comprobar = [col for col in fabricaciones_costes.columns if not fabricaciones_costes['articulo'].str.startswith('SEM').any()]
nulos = fabricaciones_costes[columnas_a_comprobar].isnull().sum()
nulos

Series([], dtype: float64)